# Tensorflow implementation 04: ResNet

by [Sho Nakagome](https://github.com/shonaka)

This jupyter notebook is intended to implement a ResNet (Residual Network) model on classification problem. We are using [Plant Seedlings Classification](https://www.kaggle.com/c/plant-seedlings-classification) as a dataset.

## Imports

In [1]:
%matplotlib inline
import tensorflow as tf
import tflearn
from tflearn.data_utils import image_preloader
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# Just to make plotting images larger on the jupyter notebook
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 16

/home/snakagom/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Check environment

In [2]:
print("Tensorflow version:\t", tf.__version__)
print("Numpy version:\t\t", np.__version__)

Tensorflow version:	 1.5.0
Numpy version:		 1.14.0


## Define global variables

In [3]:
# Image related (resizing image dimensions)
IMG_HEIGHT = 32
IMG_WIDTH = 32

# Number of classes
NUM_CLASS = 12

# Labels
LABELS = {
    0: 'Black-glass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickwead',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small=flowered Cranesbill',
    11: 'Sugar beet'}

# Optimization related
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
NUM_EPOCHS = 10

# ResNet related
# 32 layers: N=5, 56 layers: N=9, 110 layers: N=18
N = 5

In [4]:
# Specify training path
TRAIN_PATH = 'data/plant_seedlings_classification/train'
# Specify test path
TEST_PATH = 'data/plant_seedlings_classification/test'

## Define helper functions to load the data

In [5]:
trainX, trainY = image_preloader('data/plant_seedlings_classification/train',
                                 mode='folder',
                                 image_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                 categorical_labels=True,
                                 normalize=True,
                                 files_extension='.png',
                                 grayscale=True
                                )

In [36]:
imageList = os.listdir(TEST_PATH)
test_images = []
names = []
for image in imageList:
    if image[-3:] != 'png':
        continue
    names.append(image)
    new_image = Image.open(TEST_PATH+'/'+image)
    # resize
    new_image = new_image.resize([IMG_WIDTH, IMG_HEIGHT])
    # convert image to gray scale
    new_image = new_image.convert('L')
    test_images.append(new_image)

In [7]:
# Create a test set with normalization
test_imgs = np.array([np.array(img) for img in test_images])
testX = test_imgs.reshape(test_imgs.shape[0], IMG_HEIGHT, IMG_WIDTH, 1) / 255

In [37]:
names[0]

'632156793.png'

In [8]:
# Building Residual Network
net = tflearn.input_data(shape=[None, IMG_HEIGHT, IMG_WIDTH, 1])
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.residual_block(net, N, 16)
net = tflearn.residual_block(net, 1, 32, downsample=True)
net = tflearn.residual_block(net, N-1, 32)
net = tflearn.residual_block(net, 1, 64, downsample=True)
net = tflearn.residual_block(net, N-1, 64)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [9]:
# Regression
net = tflearn.fully_connected(net, NUM_CLASS, activation='softmax')
mom = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
net = tflearn.regression(net, optimizer=mom,
                         loss='categorical_crossentropy')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
# Training
model = tflearn.DNN(net,
                    max_checkpoints=10,
                    tensorboard_verbose=0)

model.fit(trainX, trainY, n_epoch=NUM_EPOCHS, validation_set=0.1,
          snapshot_epoch=False, snapshot_step=500,
          show_metric=True, batch_size=BATCH_SIZE, shuffle=True,
          run_id='resnet_plant_seedlings')

Training Step: 670  | total loss: 0.82838 | time: 38.922s
| Momentum | epoch: 010 | loss: 0.82838 - acc: 0.6981 -- iter: 4275/4275


In [11]:
prediction = model.predict(testX)

In [30]:
test_len = prediction.shape[0]
print(prediction.shape)

(794, 12)


In [13]:
print(tf.one_hot(prediction[0], NUM_CLASS))

Tensor("one_hot:0", shape=(12, 12), dtype=float32)


In [14]:
print(np.max(prediction[0]))

0.38758817


In [15]:
print(np.argmax(prediction[0]))

4


In [16]:
print(prediction[0])

[1.4995301e-01 5.8154797e-05 3.9413786e-03 1.3847487e-02 3.8758817e-01
 4.6404157e-02 1.5474094e-01 7.7142689e-04 3.6655344e-02 1.4276654e-04
 1.1789837e-04 2.0577925e-01]


In [17]:
prediction_ind = np.argmax(prediction, axis=-1)
print(prediction_ind.shape)

(794,)


In [32]:
predicted_labels = [LABELS[prediction_ind[i]] for i in range(test_len)]

In [34]:
print(predicted_labels[0])
print(predicted_labels[1])

Common wheat
Common wheat


In [38]:
df = pd.DataFrame(data={'file': names, 'species': predicted_labels})
df_sort = df.sort_values(by=['file'])
df_sort.to_csv('Results/plant_seedlings_results.csv', index=False)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
